In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow import keras
import covariate_models
import pandas as pd
import numpy as np
import os
from experiments_covariate import experiment_dict
from pathlib import Path

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import model_utils

from sklearn.metrics import r2_score

from build_datasets_covariate import *

from keras import backend as K 

## Setting Up Functions & Model

In [3]:
def assess_model(
    exp_name,
    model,
    validation_dataset,
    metrics
):
    print('-----------------------')
    

    model_history = np.load('../data/saved_models_v2/{}_history.npy'.format(exp_name), allow_pickle=True)
    model_utils.training_metrics_np_object(model_history, metrics[0])

    saved_model = '../data/saved_models_v2/{}.h5'.format(exp_name)

    model.load_weights(saved_model)    
    metric_values = model.evaluate(validation_dataset, verbose=2)
    
    preds = model.predict(validation_dataset)
    
    return preds


In [4]:
def assess_model_no_history(
    exp_name,
    model,
    validation_dataset,
    metrics
):
    print('-----------------------')
    
    saved_model = '../data/saved_models_v2/{}.h5'.format(exp_name)

    model.load_weights(saved_model)    
    metric_values = model.evaluate(validation_dataset, verbose=2)
    
    preds = model.predict(validation_dataset)
    
    return preds

In [5]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

def get_stats(
    exp_name,
    model,
    validation_dataset,
    metrics
):
    
    print('-----------------------')
    
    saved_model = '../data/saved_models_v2/{}.h5'.format(exp_name)

    model.load_weights(saved_model)        
    preds = model.predict(validation_dataset)
    
    labels = np.concatenate([x for _, x in validation_dataset])
    preds = preds.flatten()
    binary_preds = (preds > 0.5).astype(int)
    
    print('AUC:', roc_auc_score(labels, preds))
    print('Acc:', accuracy_score(labels, binary_preds))
    
    tn, fp, fn, tp = confusion_matrix(labels, binary_preds).ravel()
    
    ppv = tp / (tp + fp)
    npv = tn / (tn + fn)
    sens = tp / (tp + fn)
    spec = tn / (fp + tn)
    
    print('PPV:', ppv)
    print('NPV:', npv)
    print('Sens:', sens)
    print('Spec:', spec)

In [6]:
metrics=['binary_accuracy', 'AUC']
random_rotate_angles = []

batch_size = 1

## Evaluate Image + Covariate Models

In [8]:
for t in ['A', 'T', 'N']:

    val_gen = DataGenerator_Gaussian_Labels(
        target = '{}_val'.format(t), 
        batch_size = batch_size, 
        dim = (182, 182, 218), 
        shuffle = False,
        use_random_rotate = False,
        random_rotate_angles = random_rotate_angles
    )
    
    network = covariate_models.AD_classifier_plus_tuning(
        width=182, height=182, depth=218,
        dropout=0.5,
        image_features=100) 
    network.compile(
        loss=tf.keras.losses.binary_crossentropy, 
        metrics=metrics
    )
    
    get_stats(
        '{}_classifier_182p_tuned_gaussian_labels_param'.format(t), 
        network, 
        val_gen,
        metrics
    )
    
    K.clear_session()

-----------------------
AUC: 0.7418511066398391
Acc: 0.6312056737588653
PPV: 0.6338028169014085
NPV: 0.6285714285714286
Sens: 0.6338028169014085
Spec: 0.6285714285714286
-----------------------
AUC: 0.8289473684210527
Acc: 0.875
PPV: 0.8809523809523809
NPV: 0.8333333333333334
Sens: 0.9736842105263158
Spec: 0.5
-----------------------
AUC: 0.8404195011337868
Acc: 0.7428571428571429
PPV: 0.734375
NPV: 0.7465753424657534
Sens: 0.5595238095238095
Spec: 0.8650793650793651


## Evaluate Image Only Models

In [10]:
for t in ['A', 'T', 'N']:

    val_gen = DataGenerator_Gaussian_Labels_Image_Only(
        target = '{}_val'.format(t), 
        batch_size = batch_size, 
        dim = (182, 182, 218), 
        shuffle = False,
        use_random_rotate = False,
        random_rotate_angles = random_rotate_angles
    )
    
    network = covariate_models.AD_classifier_plus_tuning_img_only(
        width=182, height=182, depth=218,
        dropout=0.5,
        image_features=100
    ) 
    
    network.compile(
        loss=tf.keras.losses.binary_crossentropy, 
        metrics=metrics
    )
    
    get_stats(
        '{}_classifier_182p_tuned_gaussian_labeled_img_only'.format(t), 
        network, 
        val_gen,
        metrics
    )
    
    K.clear_session()

-----------------------
AUC: 0.6257545271629779
Acc: 0.624113475177305
PPV: 0.6046511627906976
NPV: 0.6545454545454545
Sens: 0.7323943661971831
Spec: 0.5142857142857142
-----------------------
AUC: 0.7605263157894737
Acc: 0.8541666666666666
PPV: 0.8444444444444444
NPV: 1.0
Sens: 1.0
Spec: 0.3
-----------------------
AUC: 0.8109410430839002
Acc: 0.7476190476190476
PPV: 0.6867469879518072
NPV: 0.7874015748031497
Sens: 0.6785714285714286
Spec: 0.7936507936507936


## Test Set

In [11]:
for t in ['A', 'T', 'N']:

    test_gen = DataGenerator_Gaussian_Labels(
        target = '{}_test'.format(t), 
        batch_size = batch_size, 
        dim = (182, 182, 218), 
        shuffle = True,
        use_random_rotate = False,
        random_rotate_angles = random_rotate_angles
    )
    
    network = covariate_models.AD_classifier_plus_tuning(
        width=182, height=182, depth=218,
        dropout=0.5,
        image_features=100) 
    network.compile(
        loss=tf.keras.losses.binary_crossentropy, 
        metrics=metrics
    )
    
    assess_model_no_history(
        '{}_classifier_182p_tuned_gaussian_labels_param'.format(t), 
        network, 
        test_gen,
        metrics
    )
    
    K.clear_session()

-----------------------
376/376 - 65s - loss: 0.5484 - binary_accuracy: 0.7128 - auc: 0.7882
-----------------------
107/107 - 19s - loss: 0.3381 - binary_accuracy: 0.8879 - auc: 0.7300
-----------------------
512/512 - 87s - loss: 0.4617 - binary_accuracy: 0.7539 - auc: 0.8567


In [12]:
for t in ['A', 'T', 'N']:

    test_gen = DataGenerator_Gaussian_Labels(
        target = '{}_test'.format(t), 
        batch_size = batch_size, 
        dim = (182, 182, 218), 
        shuffle = False,
        use_random_rotate = False,
        random_rotate_angles = random_rotate_angles
    )
    
    network = covariate_models.AD_classifier_plus_tuning(
        width=182, height=182, depth=218,
        dropout=0.5,
        image_features=100) 
    network.compile(
        loss=tf.keras.losses.binary_crossentropy, 
        metrics=metrics
    )
    
    get_stats(
        '{}_classifier_182p_tuned_gaussian_labels_param'.format(t), 
        network, 
        test_gen,
        metrics
    )
    
    K.clear_session()

-----------------------
AUC: 0.7884461924304605
Acc: 0.7127659574468085
PPV: 0.7162162162162162
NPV: 0.7077922077922078
Sens: 0.7794117647058824
Spec: 0.6337209302325582
-----------------------
AUC: 0.7291325695581015
Acc: 0.8878504672897196
PPV: 0.91
NPV: 0.5714285714285714
Sens: 0.9680851063829787
Spec: 0.3076923076923077
-----------------------
AUC: 0.8566117316254003
Acc: 0.75390625
PPV: 0.6936170212765957
NPV: 0.8050541516245487
Sens: 0.7511520737327189
Spec: 0.7559322033898305


In [14]:
for t in ['A', 'T', 'N']:

    test_gen = DataGenerator_Gaussian_Labels_Image_Only(
        target = '{}_test'.format(t), 
        batch_size = batch_size, 
        dim = (182, 182, 218), 
        shuffle = False,
        use_random_rotate = False,
        random_rotate_angles = random_rotate_angles
    )
    
    network = covariate_models.AD_classifier_plus_tuning_img_only(
        width=182, height=182, depth=218,
        dropout=0.5,
        image_features=100) 
    network.compile(
        loss=tf.keras.losses.binary_crossentropy, 
        metrics=metrics
    )
    
    get_stats(
        '{}_classifier_182p_tuned_gaussian_labeled_img_only'.format(t), 
        network, 
        test_gen,
        metrics
    )
    
    K.clear_session()

-----------------------
AUC: 0.7331566347469219
Acc: 0.6622340425531915
PPV: 0.6558704453441295
NPV: 0.6744186046511628
Sens: 0.7941176470588235
Spec: 0.5058139534883721
-----------------------
AUC: 0.8011456628477904
Acc: 0.8691588785046729
PPV: 0.8921568627450981
NPV: 0.4
Sens: 0.9680851063829787
Spec: 0.15384615384615385
-----------------------
AUC: 0.8498164492697022
Acc: 0.7578125
PPV: 0.6882591093117408
NPV: 0.8226415094339623
Sens: 0.783410138248848
Spec: 0.7389830508474576
